In [1]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
import mlflow
import pandas as pd
import numpy as np
from prefect import task, Flow
from datetime import timedelta
from prefect.schedules import IntervalSchedule

import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import mlflow.pyfunc
from prefect import task, Flow, Parameter, Client
from prefect.run_configs import KubernetesRun
from prefect.schedules import IntervalSchedule
from prefect.storage import S3
from prefect.tasks.control_flow import merge
from prefect import task, Flow, case
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from datetime import timedelta
import numpy as np
import pandas as pd
import mlflow
import requests
from typing import Tuple
import logging
import h5py
import sqlite3
import time

In [6]:

def calculate_rmse():
    
    filename = "best_predictions.h5"

    with h5py.File(filename, "r") as f:
        # Print all root level object names (aka keys) 
        # these can be group or dataset names 
        print("Keys: %s" % f.keys())
        # get first object name/key; may or may NOT be a group
        a_group_key = list(f.keys())[0]

        # get the object type for a_group_key: usually group or dataset
        print(type(f[a_group_key])) 

        # If a_group_key is a group name, 
        # this gets the object names in the group and returns as a list
        data = list(f[a_group_key])

        # If a_group_key is a dataset name, 
        # this gets the dataset values and returns as a list
        data = list(f[a_group_key])
        # preferred methods to get dataset values:
        ds_obj = f[a_group_key]      # returns as a h5py dataset object
        ds_arr = f[a_group_key][()]  # returns as a numpy array
        
        # Use True lable to calculate RMSE
        return 0.7
    


In [7]:
@task
def is_model_degraded_condition():
    rmse = calculate_rmse()
    if rmse > 0.4:
        return True
    else:
        return False  

    
@task
def inference():
    os.system("sbatch ./../../inference_harsh_cp_1.sh")    
    time.sleep(30)
    
@task
def save_to_database():
    pass

 

In [9]:
schedule = IntervalSchedule(interval=timedelta(minutes=2))


# 1. Get fresh data
# 2. Pre-process the data
# 3. Do a inference with that
# 4. If model is working above a set threshold. Fine. 
# 5. If not, then train the model on new data.
# 6. Evaluate the model and check if it is still fine or not and take specific action

# TODO:
# in case of failure of any task, retry 
# save the results in database

with Flow("MLOps", schedule=schedule) as flow:
    
    # download_data()
    # pre_process()
    
    inference()
    
    
    # read the results and tell if the model is degraded
    is_model_degraded = is_model_degraded_condition()

    # train on new data if error is more than threshold and log the results on test data
    with case(is_model_degraded, True): 
#         model = train_model_task(train_data)        
        inference()
        is_model_degraded_condition()

    # save_to_db()
    
state = flow.run()
print('DONE')
# flow.visualize()

[2022-08-25 13:21:06+0200] INFO - prefect.FlowRunner | Beginning Flow run for 'MLOps'
[2022-08-25 13:21:06+0200] INFO - prefect.TaskRunner | Task 'inference': Starting task run...
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'inference': Finished task run for task with final state: 'Success'
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'is_model_degraded_condition': Starting task run...
Keys: <KeysViewHDF5 ['y_pred']>
<class 'h5py._hl.dataset.Dataset'>
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'is_model_degraded_condition': Finished task run for task with final state: 'Success'
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'case(True)': Starting task run...
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'case(True)': Finished task run for task with final state: 'Success'
[2022-08-25 13:21:37+0200] INFO - prefect.TaskRunner | Task 'inference': Starting task run...
[2022-08-25 13:22:07+0200] INFO - prefect.TaskRun